### **1. Planificación**

El modelo permitirá predecir la demanda de pasajeros de las estaciones en base a la data histórica previamente recolectada, lo que ayudará a mejorar la experiencia de los usuarios, reduciendo la congestión y los tiempos de espera.

### **2. Obtención de data**

#### **1.1. Instalación de librerías**

In [1]:
%pip install prophet
%pip install pandas
%pip install openpyxl
%pip install numpy 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


#### **1.2. Importar librerías**

In [2]:
import pandas as pd
import openpyxl
import numpy as np

In [3]:
from openpyxl import load_workbook
from prophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

c:\Users\annsh\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


#### **1.3. Lectura de data**

In [4]:
# Ruta del archivo Excel
file_path_2019 = 'C:\\Users\\annsh\\Desktop\\ANNSHERLY\\10 CICLO\\TP1\\Modelo predictivo- data y pruebas\\Modelo\\Datasets\\2019.xlsx'  
file_path_2022 = 'C:\\Users\\annsh\\Desktop\\ANNSHERLY\\10 CICLO\\TP1\\Modelo predictivo- data y pruebas\\Modelo\\Datasets\\2022.xlsx' 
file_path_2023 = 'C:\\Users\\annsh\\Desktop\\ANNSHERLY\\10 CICLO\\TP1\\Modelo predictivo- data y pruebas\\Modelo\\Datasets\\2023.xlsx'  
file_path_2024 = 'C:\\Users\\annsh\\Desktop\\ANNSHERLY\\10 CICLO\\TP1\\Modelo predictivo- data y pruebas\\Modelo\\Datasets\\2024.xlsx' 
file_path_2025 = 'C:\\Users\\annsh\\Desktop\\ANNSHERLY\\10 CICLO\\TP1\\Modelo predictivo- data y pruebas\\Modelo\\Datasets\\2025.xlsx' 
# Leer el archivo Excel
df1 = pd.read_excel(file_path_2019, sheet_name='Sheet1')  
df2= pd.read_excel(file_path_2022, sheet_name='Sheet1') 
df3 = pd.read_excel(file_path_2023, sheet_name='Sheet1') 
df4 = pd.read_excel(file_path_2024, sheet_name='Sheet1') 
df5 = pd.read_excel(file_path_2025, sheet_name='Sheet1') 


In [5]:
# Combinamos los DataFrames en uno solo
df_unificado = pd.concat([df1, df4, df2, df3, df5], ignore_index=True)
df_unificado.head()

,Año,Mes,Periodo,Entidad Prestadora,Concesión,Estación,Fecha,Rango Horario,Tipo Pasajero,Tipo Tarifa,Tipo Pase,Número de Pasajeros,Entidad prestadora
0,2019,Enero,201901,GYM FERROVÍAS S.A.,TL1,1.- VILLA EL SALVADOR,2019-01-01,6-7 h,NORMAL,Intervalos en tarifa Adulto,00 Normal,472,NaN
1,2019,Enero,201901,GYM FERROVÍAS S.A.,TL1,1.- VILLA EL SALVADOR,2019-01-01,6-7 h,NORMAL,Intervalos en tarifa Universitario / Medio,00 Normal,0,NaN
2,2019,Enero,201901,GYM FERROVÍAS S.A.,TL1,1.- VILLA EL SALVADOR,2019-01-01,7-8 h,NORMAL,Intervalos en tarifa Adulto,00 Normal,362,NaN
3,2019,Enero,201901,GYM FERROVÍAS S.A.,TL1,1.- VILLA EL SALVADOR,2019-01-01,7-8 h,NORMAL,Intervalos en tarifa Universitario / Medio,00 Normal,0,NaN
4,2019,Enero,201901,GYM FERROVÍAS S.A.,TL1,1.- VILLA EL SALVADOR,2019-01-01,8-9 h,NORMAL,Intervalos en tarifa Adulto,00 Normal,351,NaN


### **3. Limpieza de data**

#### **3.1. Eliminación de valores nulos**

In [6]:
df_unificado.dropna(subset=['Rango Horario', 'Fecha', 'Número de Pasajeros'], inplace=True)

#### **3.4. Preparación de data**

In [7]:
# Aseguramos que los tipos de datos sean correctos
df_unificado['Año'] = df_unificado['Año'].astype(int)
df_unificado['Mes'] = df_unificado['Mes'].astype(str)
df_unificado['Periodo'] = df_unificado['Periodo'].astype(int)
df_unificado['Entidad Prestadora'] = df_unificado['Entidad Prestadora'].astype(str)
df_unificado['Concesión'] = df_unificado['Concesión'].astype(str)
df_unificado['Estación'] = df_unificado['Estación'].astype(str)
df_unificado['Fecha'] = pd.to_datetime(df_unificado['Fecha'], format='%Y-%m-%d')
df_unificado['Rango Horario'] = df_unificado['Rango Horario'].astype(str)
df_unificado['Tipo Pasajero'] = df_unificado['Tipo Pasajero'].astype(str)
df_unificado['Tipo Tarifa'] = df_unificado['Tipo Tarifa'].astype(str)
df_unificado['Tipo Pase'] = df_unificado['Tipo Pase'].astype(str)
df_unificado['Número de Pasajeros'] = df_unificado['Número de Pasajeros'].astype(int)

In [8]:
# Preprocesamos los datos para Prophet
df_unificado['Hora_Inicio'] = df_unificado['Rango Horario'].apply(lambda x: f'{int(x.split("-")[0].strip()):02d}:00')
df_unificado['ds'] = pd.to_datetime(df_unificado['Fecha'].dt.strftime('%Y-%m-%d') + ' ' + df_unificado['Hora_Inicio'], format='%Y-%m-%d %H:%M')
df_unificado['y'] = df_unificado['Número de Pasajeros']

In [9]:
print(df_unificado.columns)

Index(['Año', 'Mes', 'Periodo', 'Entidad Prestadora', 'Concesión', 'Estación',
       'Fecha', 'Rango Horario', 'Tipo Pasajero', 'Tipo Tarifa', 'Tipo Pase',
       'Número de Pasajeros', 'Entidad prestadora', 'Hora_Inicio', 'ds', 'y'],
      dtype='object')


In [10]:
df_unificado.head()

,Año,Mes,Periodo,Entidad Prestadora,Concesión,Estación,Fecha,Rango Horario,Tipo Pasajero,Tipo Tarifa,Tipo Pase,Número de Pasajeros,Entidad prestadora,Hora_Inicio,ds,y
0,2019,Enero,201901,GYM FERROVÍAS S.A.,TL1,1.- VILLA EL SALVADOR,2019-01-01,6-7 h,NORMAL,Intervalos en tarifa Adulto,00 Normal,472,NaN,06:00,2019-01-01 06:00:00,472
1,2019,Enero,201901,GYM FERROVÍAS S.A.,TL1,1.- VILLA EL SALVADOR,2019-01-01,6-7 h,NORMAL,Intervalos en tarifa Universitario / Medio,00 Normal,0,NaN,06:00,2019-01-01 06:00:00,0
2,2019,Enero,201901,GYM FERROVÍAS S.A.,TL1,1.- VILLA EL SALVADOR,2019-01-01,7-8 h,NORMAL,Intervalos en tarifa Adulto,00 Normal,362,NaN,07:00,2019-01-01 07:00:00,362
3,2019,Enero,201901,GYM FERROVÍAS S.A.,TL1,1.- VILLA EL SALVADOR,2019-01-01,7-8 h,NORMAL,Intervalos en tarifa Universitario / Medio,00 Normal,0,NaN,07:00,2019-01-01 07:00:00,0
4,2019,Enero,201901,GYM FERROVÍAS S.A.,TL1,1.- VILLA EL SALVADOR,2019-01-01,8-9 h,NORMAL,Intervalos en tarifa Adulto,00 Normal,351,NaN,08:00,2019-01-01 08:00:00,351


In [11]:
# Nos aseguramos de que las columnas necesarias estén en el formato correcto
df_unificado['Hora_Inicio'] = pd.to_datetime(df_unificado['Hora_Inicio'])
df_unificado['ds'] = pd.to_datetime(df_unificado['ds'])  # Esto es para asegurar el correcto formato de 'ds'

# Agrupamos por las columnas que identifican una hora específica, sumando el número de pasajeros
df_agrupado = df_unificado.groupby(['Fecha', 'Estación', 'Hora_Inicio', 'ds']).agg({
    'Número de Pasajeros': 'sum',
    'y': 'sum'
}).reset_index()


df_agrupado.head()


C:\Users\annsh\AppData\Local\Temp\ipykernel_14668\1600994336.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_unificado['Hora_Inicio'] = pd.to_datetime(df_unificado['Hora_Inicio'])


,Fecha,Estación,Hora_Inicio,ds,Número de Pasajeros,y
0,2019-01-01,1.- VILLA EL SALVADOR,2025-06-22 06:00:00,2019-01-01 06:00:00,472,472
1,2019-01-01,1.- VILLA EL SALVADOR,2025-06-22 07:00:00,2019-01-01 07:00:00,362,362
2,2019-01-01,1.- VILLA EL SALVADOR,2025-06-22 08:00:00,2019-01-01 08:00:00,351,351
3,2019-01-01,1.- VILLA EL SALVADOR,2025-06-22 09:00:00,2019-01-01 09:00:00,380,380
4,2019-01-01,1.- VILLA EL SALVADOR,2025-06-22 10:00:00,2019-01-01 10:00:00,490,490


### **4. Entrenamiento del modelo**

In [16]:
from datetime import datetime, timedelta

In [19]:
from datetime import datetime, timedelta

print("🚀 Iniciando entrenamiento del modelo optimizado...")

# Creamos listas para almacenar las predicciones y métricas
predicciones_excel = []
metricas_estaciones = []

# Obtenemos las estaciones únicas
estaciones = df_agrupado['Estación'].unique()
print(f"📊 Total de estaciones a procesar: {len(estaciones)}")

# Creamos un ID secuencial
contador_id = 1

# Definimos el rango de fechas para predicción (2 meses)
fecha_inicio_pred = '2025-06-01'
fecha_fin_pred = '2025-07-31'
print(f"📅 Horizonte de predicción: {fecha_inicio_pred} a {fecha_fin_pred}")

# Configuración de progreso
total_estaciones = len(estaciones)
estaciones_procesadas = 0

# Iteramos sobre cada estación
for estacion in estaciones:
    estaciones_procesadas += 1
    print(f"\n{'='*60}")
    print(f"📍 [{estaciones_procesadas}/{total_estaciones}] Procesando: {estacion}")
    print(f"{'='*60}")
    
    try:
        # Filtramos los datos por la estación actual
        df_estacion = df_agrupado[df_agrupado['Estación'] == estacion].copy()
        
        # Verificamos que hay suficientes datos
        if len(df_estacion) < 50:
            print(f"⚠️  Datos insuficientes para {estacion} ({len(df_estacion)} registros). Saltando...")
            continue
        
        # Ordenamos los datos por fecha
        df_estacion = df_estacion.sort_values('ds')
        
        print(f"�� Datos disponibles: {len(df_estacion)} registros")
        print(f"📅 Rango temporal: {df_estacion['ds'].min()} a {df_estacion['ds'].max()}")
        
        # Agregamos regresores adicionales considerando toda la data histórica
        df_estacion['día_semana'] = df_estacion['ds'].dt.dayofweek
        df_estacion['es_fin_semana'] = df_estacion['día_semana'].isin([5, 6]).astype(int)
        df_estacion['hora_día'] = df_estacion['ds'].dt.hour
        df_estacion['mes'] = df_estacion['ds'].dt.month
        df_estacion['año'] = df_estacion['ds'].dt.year
        df_estacion['día_año'] = df_estacion['ds'].dt.dayofyear  # Para patrones anuales
        
        # Detección y tratamiento de outliers usando IQR
        Q1 = df_estacion['y'].quantile(0.25)
        Q3 = df_estacion['y'].quantile(0.75)
        IQR = Q3 - Q1
        outliers_mask = (df_estacion['y'] < (Q1 - 1.5 * IQR)) | (df_estacion['y'] > (Q3 + 1.5 * IQR))
        outliers_count = outliers_mask.sum()
        
        if outliers_count > 0:
            print(f"🔍 Detectados {outliers_count} outliers. Aplicando filtro IQR...")
            df_estacion = df_estacion[~outliers_mask]
        
        # Dividimos los datos en entrenamiento y validación (últimos 2 meses para validación)
        fecha_corte = df_estacion['ds'].max() - timedelta(days=60)
        train_data = df_estacion[df_estacion['ds'] < fecha_corte]
        val_data = df_estacion[df_estacion['ds'] >= fecha_corte]
        
        print(f"🏋️  Datos de entrenamiento: {len(train_data)} registros")
        print(f"📊 Datos de validación: {len(val_data)} registros")
        
        if len(train_data) < 30:
            print(f"⚠️  Datos de entrenamiento insuficientes. Saltando...")
            continue
        
        # Configuramos el modelo con parámetros optimizados
        print("⚙️  Configurando modelo Prophet...")
        modelo = Prophet(
            changepoint_prior_scale=0.05,      # Controla la flexibilidad de la tendencia
            seasonality_prior_scale=10.0,      # Controla la fuerza de la estacionalidad
            seasonality_mode='multiplicative', # Mejor para datos con estacionalidad que crece
            yearly_seasonality=True,           # Incluye estacionalidad anual (patrones por año)
            weekly_seasonality=True,           # Incluye estacionalidad semanal
            daily_seasonality=True,            # Incluye estacionalidad diaria (por hora)
            interval_width=0.95                # Intervalo de confianza del 95%
        )
        
        # Agregamos regresores al modelo
        modelo.add_regressor('día_semana')
        modelo.add_regressor('es_fin_semana')
        modelo.add_regressor('hora_día')
        modelo.add_regressor('mes')
        modelo.add_regressor('día_año')  # Para capturar patrones anuales específicos
        
        # Entrenamos el modelo con TODA la data histórica disponible
        print("🏋️  Entrenando modelo...")
        modelo.fit(train_data[['ds', 'y', 'día_semana', 'es_fin_semana', 'hora_día', 'mes', 'día_año']])
        print("✅ Modelo entrenado exitosamente")
        
        # VALIDACIÓN SIMPLIFICADA Y EFICIENTE
        print("📊 Calculando métricas de validación...")
        
        if len(val_data) > 0:
            # Predicción en datos de validación
            futuro_val = val_data[['ds', 'día_semana', 'es_fin_semana', 'hora_día', 'mes', 'día_año']].copy()
            pred_val = modelo.predict(futuro_val)
            
            # Valores reales y predichos
            y_true = val_data['y'].values
            y_pred = pred_val['yhat'].values
            
            # Calculamos métricas de precisión
            mae = mean_absolute_error(y_true, y_pred)
            rmse = np.sqrt(mean_squared_error(y_true, y_pred))
            r2 = r2_score(y_true, y_pred)
            mape = np.mean(np.abs((y_true - y_pred) / np.where(y_true != 0, y_true, 1))) * 100
            
            print(f"📈 Métricas calculadas:")
            print(f"   - MAE: {mae:.2f}")
            print(f"   - RMSE: {rmse:.2f}")
            print(f"   - R²: {r2:.4f}")
            print(f"   - MAPE: {mape:.2f}%")
        else:
            mae = rmse = r2 = mape = 0
            print("⚠️  No hay datos de validación disponibles")
        
        # Guardamos las métricas
        metricas_estaciones.append({
            'Estación': estacion,
            'MAE': mae,
            'RMSE': rmse,
            'R2': r2,
            'MAPE': mape,
            'Datos_Entrenamiento': len(train_data),
            'Datos_Validacion': len(val_data),
            'Outliers_Eliminados': outliers_count
        })
        
        # Generamos fechas futuras para predicción (por hora)
        print("🔮 Generando predicciones futuras...")
        fechas_futuras = pd.date_range(start=fecha_inicio_pred, end=fecha_fin_pred, freq='H')
        # Filtramos solo horas de operación (5:00 AM a 11:00 PM)
        fechas_futuras = fechas_futuras[(fechas_futuras.hour >= 5) & (fechas_futuras.hour <= 23)]
        
        print(f"📅 Generando {len(fechas_futuras)} predicciones por hora...")
        
        # Preparamos el DataFrame para predicción
        futuro = pd.DataFrame({
            'ds': fechas_futuras,
            'día_semana': fechas_futuras.dayofweek,
            'es_fin_semana': fechas_futuras.dayofweek.isin([5, 6]).astype(int),
            'hora_día': fechas_futuras.hour,
            'mes': fechas_futuras.month,
            'día_año': fechas_futuras.dayofyear
        })
        
        # Realizamos las predicciones
        predicciones = modelo.predict(futuro)
        
        # Ajustamos las predicciones (valores no negativos y enteros)
        predicciones['yhat'] = predicciones['yhat'].round().astype(int).apply(lambda x: max(0, x))
        predicciones['yhat_lower'] = predicciones['yhat_lower'].round().astype(int).apply(lambda x: max(0, x))
        predicciones['yhat_upper'] = predicciones['yhat_upper'].round().astype(int).apply(lambda x: max(0, x))
        
        # Guardamos las predicciones
        print("💾 Guardando predicciones...")
        for idx, row in predicciones.iterrows():
            predicciones_excel.append({
                'ID': contador_id,
                'Fecha': row['ds'].date(),
                'Hora': row['ds'].time(),
                'Fecha_y_Hora': row['ds'],
                'Estación': estacion,
                'Predicción': row['yhat'],
                'Límite_Inferior': row['yhat_lower'],
                'Límite_Superior': row['yhat_upper'],
                'Día_Semana': row['ds'].strftime('%A'),
                'Es_Fin_Semana': 'Sí' if row['ds'].dayofweek in [5, 6] else 'No',
                'MAE': mae,
                'RMSE': rmse,
                'R2': r2,
                'MAPE': mape
            })
            contador_id += 1
        
        print(f"✅ {len(predicciones)} predicciones generadas para {estacion}")
        
    except Exception as e:
        print(f"❌ Error procesando {estacion}: {str(e)}")
        continue

print(f"\n{'='*60}")
print(f"�� PROCESAMIENTO COMPLETADO")
print(f"{'='*60}")
print(f"📊 Estaciones procesadas: {estaciones_procesadas}/{total_estaciones}")
print(f"🔮 Total de predicciones generadas: {len(predicciones_excel)}")

# Convertimos las predicciones a DataFrame
if predicciones_excel:
    df_predicciones = pd.DataFrame(predicciones_excel)
    
    # Guardamos las predicciones en Excel
    nombre_predicciones = 'Predicciones-01_06_2025_2meses_mejorado.xlsx'
    df_predicciones.to_excel(nombre_predicciones, index=False)
    print(f"💾 Predicciones guardadas en: {nombre_predicciones}")
    
    # Guardamos las métricas por estación
    df_metricas = pd.DataFrame(metricas_estaciones)
    nombre_metricas = 'metricas_estaciones_01_06_2025_2meses_mejorado.xlsx'
    df_metricas.to_excel(nombre_metricas, index=False)
    print(f"📊 Métricas guardadas en: {nombre_metricas}")
    
    # Mostramos un resumen de las métricas
    print(f"\n{'='*60}")
    print("📈 RESUMEN DE MÉTRICAS POR ESTACIÓN")
    print(f"{'='*60}")
    print(df_metricas.to_string(index=False))
    
    # Estadísticas generales
    print(f"\n{'='*60}")
    print("📊 ESTADÍSTICAS GENERALES")
    print(f"{'='*60}")
    print(f"Promedio MAE: {df_metricas['MAE'].mean():.2f}")
    print(f"Promedio RMSE: {df_metricas['RMSE'].mean():.2f}")
    print(f"Promedio R²: {df_metricas['R2'].mean():.4f}")
    print(f"Promedio MAPE: {df_metricas['MAPE'].mean():.2f}%")
    
    # Mostramos primeras predicciones
    print(f"\n{'='*60}")
    print("🔮 PRIMERAS 10 PREDICCIONES")
    print(f"{'='*60}")
    print(df_predicciones[['ID', 'Fecha', 'Hora', 'Estación', 'Predicción', 'R2']].head(10).to_string(index=False))
    
else:
    print("❌ No se generaron predicciones")

🚀 Iniciando entrenamiento del modelo optimizado...
📊 Total de estaciones a procesar: 26
📅 Horizonte de predicción: 2025-06-01 a 2025-07-31

📍 [1/26] Procesando: 1.- VILLA EL SALVADOR
�� Datos disponibles: 27250 registros
📅 Rango temporal: 2019-01-01 06:00:00 a 2025-03-31 22:00:00
🔍 Detectados 2879 outliers. Aplicando filtro IQR...
🏋️  Datos de entrenamiento: 23445 registros
📊 Datos de validación: 926 registros
⚙️  Configurando modelo Prophet...
🏋️  Entrenando modelo...


01:19:23 - cmdstanpy - INFO - Chain [1] start processing
01:19:31 - cmdstanpy - INFO - Chain [1] done processing


✅ Modelo entrenado exitosamente
📊 Calculando métricas de validación...
📈 Métricas calculadas:
   - MAE: 235.21
   - RMSE: 303.78
   - R²: 0.8096
   - MAPE: 646.29%
🔮 Generando predicciones futuras...


C:\Users\annsh\AppData\Local\Temp\ipykernel_14668\3357931779.py:143: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  fechas_futuras = pd.date_range(start=fecha_inicio_pred, end=fecha_fin_pred, freq='H')


📅 Generando 1140 predicciones por hora...
💾 Guardando predicciones...
✅ 1140 predicciones generadas para 1.- VILLA EL SALVADOR

📍 [2/26] Procesando: 10.- LOS CABITOS
�� Datos disponibles: 27250 registros
📅 Rango temporal: 2019-01-01 06:00:00 a 2025-03-31 22:00:00
🔍 Detectados 2334 outliers. Aplicando filtro IQR...
🏋️  Datos de entrenamiento: 23961 registros
📊 Datos de validación: 955 registros
⚙️  Configurando modelo Prophet...
🏋️  Entrenando modelo...


...

💾 Guardando predicciones...
✅ 1140 predicciones generadas para 8.- JORGE CHÁVEZ

📍 [26/26] Procesando: 9.- AYACUCHO
�� Datos disponibles: 27250 registros
📅 Rango temporal: 2019-01-01 06:00:00 a 2025-03-31 22:00:00
🔍 Detectados 2096 outliers. Aplicando filtro IQR...
🏋️  Datos de entrenamiento: 24199 registros
📊 Datos de validación: 955 registros
⚙️  Configurando modelo Prophet...
🏋️  Entrenando modelo...


01:28:34 - cmdstanpy - INFO - Chain [1] start processing
01:28:45 - cmdstanpy - INFO - Chain [1] done processing


✅ Modelo entrenado exitosamente
📊 Calculando métricas de validación...
📈 Métricas calculadas:
   - MAE: 90.36
   - RMSE: 124.51
   - R²: 0.7370
   - MAPE: 22.84%
🔮 Generando predicciones futuras...
📅 Generando 1140 predicciones por hora...


C:\Users\annsh\AppData\Local\Temp\ipykernel_14668\3357931779.py:143: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  fechas_futuras = pd.date_range(start=fecha_inicio_pred, end=fecha_fin_pred, freq='H')


💾 Guardando predicciones...
✅ 1140 predicciones generadas para 9.- AYACUCHO

�� PROCESAMIENTO COMPLETADO
📊 Estaciones procesadas: 26/26
🔮 Total de predicciones generadas: 29640
💾 Predicciones guardadas en: Predicciones-01_06_2025_2meses_mejorado.xlsx
📊 Métricas guardadas en: metricas_estaciones_01_06_2025_2meses_mejorado.xlsx

📈 RESUMEN DE MÉTRICAS POR ESTACIÓN
               Estación        MAE       RMSE       R2       MAPE  Datos_Entrenamiento  Datos_Validacion  Outliers_Eliminados
  1.- VILLA EL SALVADOR 235.206344 303.781252 0.809551 646.291475                23445               926                 2879
       10.- LOS CABITOS 148.551455 242.241555 0.819387 435.777547                23961               955                 2334
           11.- ANGAMOS 253.279269 393.300798 0.799022 654.337422                24847               995                 1408
     12.- SAN BORJA SUR  80.375805 125.838967 0.766109 233.373259                23423               943                 2884
      